In [455]:
import numpy as np
import pandas as pd

In [467]:
df = pd.read_csv('data/df_with_neis_details.csv')
df
df['start_dt'] = pd.to_datetime(df['start_dt'], errors='coerce')
df['end_dt'] = pd.to_datetime(df['end_dt'],errors='coerce')

In [487]:
def hour_range_list(end_dt,next_start_dt,start_dt,dest_nei_id, origin_nei_id):
    l = pd.period_range(end_dt, next_start_dt, freq='H').values.tolist()
    if origin_nei_id == dest_nei_id and start_dt in l:
        l.remove(start_dt)
    if next_start_dt in l:
        l.remove(next_start_dt)
    return l

In [488]:
def diff_stay(dest_nei_id, origin_nei_id, end_dt, next_start_dt, start_dt):
    if origin_nei_id == dest_nei_id and end_dt.hour == start_dt.hour:
        return []
    else:
        return [end_dt]
    
    if origin_nei_id == dest_nei_id and end_dt.hour != start_dt.hour:
        return [end_dt]
    elif origin_nei_id != dest_nei_id and end_dt.hour != start_dt.hour:
        return [end_dt]
    elif origin_nei_id != dest_nei_id and end_dt.hour == start_dt.hour:
        return [end_dt]

In [485]:
gb = df.groupby('vehicle_id')
groups_vehicle_id = [gb.get_group(x).sort_values(by=['start_dt']) for x in gb.groups]
groups_vehicle_id_shift = [gb.get_group(x).shift(periods=1).sort_values(by=['start_dt']) for x in gb.groups]

In [517]:
l_explode = []
for g in groups_vehicle_id:
    g_shift = g.shift(periods=-1).sort_values(by=['start_dt'])
    g["next_origin_nei_id"] = g_shift["origin_nei_id"]
    g["next_start_dt"] = g_shift["start_dt"]
    g['hour_list'] = g.apply(lambda x: hour_range_list(x.end_dt, x.next_start_dt, x.start_dt,x.dest_nei_id, x.origin_nei_id) if x.dest_nei_id == x.next_origin_nei_id else diff_stay(x.dest_nei_id, x.origin_nei_id, x.end_dt, x.next_start_dt, x.start_dt), axis=1)
    l_explode.append(g.explode('hour_list'))

In [519]:
k = l_explode[191][["start_dt","origin_nei_id" ,"end_dt","dest_nei_id", "next_start_dt", "next_origin_nei_id","hour_list","start_hour"]]
k.head(10)

,start_dt,origin_nei_id,end_dt,dest_nei_id,next_start_dt,next_origin_nei_id,hour_list,start_hour
49666,2017-10-23 07:46:54,58.0,2017-10-23 08:47:42.353,58.0,2017-10-23 19:11:05,58.0,2017-10-23 08:00,7
49666,2017-10-23 07:46:54,58.0,2017-10-23 08:47:42.353,58.0,2017-10-23 19:11:05,58.0,2017-10-23 09:00,7
49666,2017-10-23 07:46:54,58.0,2017-10-23 08:47:42.353,58.0,2017-10-23 19:11:05,58.0,2017-10-23 10:00,7
49666,2017-10-23 07:46:54,58.0,2017-10-23 08:47:42.353,58.0,2017-10-23 19:11:05,58.0,2017-10-23 11:00,7
49666,2017-10-23 07:46:54,58.0,2017-10-23 08:47:42.353,58.0,2017-10-23 19:11:05,58.0,2017-10-23 12:00,7
49666,2017-10-23 07:46:54,58.0,2017-10-23 08:47:42.353,58.0,2017-10-23 19:11:05,58.0,2017-10-23 13:00,7
49666,2017-10-23 07:46:54,58.0,2017-10-23 08:47:42.353,58.0,2017-10-23 19:11:05,58.0,2017-10-23 14:00,7
49666,2017-10-23 07:46:54,58.0,2017-10-23 08:47:42.353,58.0,2017-10-23 19:11:05,58.0,2017-10-23 15:00,7
49666,2017-10-23 07:46:54,58.0,2017-10-23 08:47:42.353,58.0,2017-10-23 19:11:05,58.0,2017-10-23 16:00,7
49666,2017-10-23 07:46:54,58.0,2017-10-23 08:47:42.353,58.0,2017-10-23 19:11:05,58.0,2017-10-23 17:00,7


In [512]:
k = groups_vehicle_id[191][["start_dt","origin_nei_id" ,"end_dt","dest_nei_id", "next_start_dt", "next_origin_nei_id","hour_list","start_hour"]]
k.head(10)


,start_dt,origin_nei_id,end_dt,dest_nei_id,next_start_dt,next_origin_nei_id,hour_list,start_hour
49666,2017-10-23 07:46:54,58.0,2017-10-23 08:47:42.353,58.0,2017-10-23 19:11:05,58.0,"[2017-10-23 08:00, 2017-10-23 09:00, 2017-10-2...",7
49669,2017-10-23 19:11:05,58.0,2017-10-23 23:06:48.320,58.0,2017-10-24 09:18:09,58.0,"[2017-10-23 23:00, 2017-10-24 00:00, 2017-10-2...",19
96082,2017-10-24 09:18:09,58.0,2017-10-24 09:42:43.217,40.0,2017-10-24 15:42:15,19.0,[2017-10-24 09:42:43.217000],9
65119,2017-10-24 15:42:15,19.0,2017-10-24 16:11:53.107,30.0,2017-10-24 16:56:00,30.0,[2017-10-24 16:00],15
104914,2017-10-24 16:56:00,30.0,2017-10-24 17:18:24.760,19.0,2017-10-25 10:00:26,19.0,"[2017-10-24 17:00, 2017-10-24 18:00, 2017-10-2...",16
43166,2017-10-25 10:00:26,19.0,2017-10-25 10:23:29.287,10.0,2017-10-26 06:50:28,40.0,[2017-10-25 10:23:29.287000],10
103829,2017-10-26 06:50:28,40.0,2017-10-26 07:01:41.947,35.0,2017-10-26 07:16:33,35.0,[2017-10-26 07:00],6
45215,2017-10-26 07:16:33,35.0,2017-10-26 07:40:49.307,11.0,2017-10-26 07:43:25,11.0,[2017-10-26 07:00],7
44450,2017-10-26 07:43:25,11.0,2017-10-26 07:45:01.940,11.0,2017-10-26 11:15:49,11.0,"[2017-10-26 07:00, 2017-10-26 08:00, 2017-10-2...",7
69060,2017-10-26 11:15:49,11.0,2017-10-26 11:31:14.693,34.0,2017-10-26 16:43:28,34.0,"[2017-10-26 11:00, 2017-10-26 12:00, 2017-10-2...",11


In [522]:
r = pd.concat(l_explode)
r = r.sort_index()
r

,vehicle_id,order_id,shiryun_cat,avg_dist,billing_minutes,orig_addr,dest_addr,start_dt,end_dt,kmh,...,orig_nei_name,orig_nei_area,orig_nei_popu,dest_nei_id,dest_nei_name,dest_nei_area,dest_nei_popu,next_origin_nei_id,next_start_dt,hour_list
0,46-525-81,60386,שריון חינמי,8,12.92,"גורדון 86-88, תל אביב יפו, ישראל","הרב רובינשטיין 61, תל אביב יפו, ישראל",2017-10-23 00:08:53,2017-10-23 00:31:40.370,37.151703,...,הצפון הישן-החלק הדרומי,1.348333e+06,29394,49.0,(יפו ד' (גבעת התמרים,389591.6863,6631,35.0,2017-10-24 15:58:11,2017-10-23 00:31:40.370000
1,46-549-81,60953,שריון חינמי,6,19.05,"שדרות מסריק 19, תל אביב יפו, ישראל","הרב רובינשטיין 63, תל אביב יפו, ישראל",2017-10-24 04:01:12,2017-10-24 04:24:45.240,18.897638,...,הצפון הישן-החלק הדרומי,1.348333e+06,29394,49.0,(יפו ד' (גבעת התמרים,389591.6863,6631,46.0,2017-10-28 20:58:25,2017-10-24 04:24:45.240000
2,60-609-55,66006,שריון חינמי,8,13.12,"נצח ישראל 6, תל אביב יפו, ישראל","סומקן 34, תל אביב יפו, ישראל",2017-11-02 02:19:45,2017-11-02 02:41:30.513,36.585366,...,הצפון הישן-החלק הדרומי,1.348333e+06,29394,49.0,(יפו ד' (גבעת התמרים,389591.6863,6631,49.0,2017-11-02 20:26:47,2017-11-02 05:00
2,60-609-55,66006,שריון חינמי,8,13.12,"נצח ישראל 6, תל אביב יפו, ישראל","סומקן 34, תל אביב יפו, ישראל",2017-11-02 02:19:45,2017-11-02 02:41:30.513,36.585366,...,הצפון הישן-החלק הדרומי,1.348333e+06,29394,49.0,(יפו ד' (גבעת התמרים,389591.6863,6631,49.0,2017-11-02 20:26:47,2017-11-02 20:00
2,60-609-55,66006,שריון חינמי,8,13.12,"נצח ישראל 6, תל אביב יפו, ישראל","סומקן 34, תל אביב יפו, ישראל",2017-11-02 02:19:45,2017-11-02 02:41:30.513,36.585366,...,הצפון הישן-החלק הדרומי,1.348333e+06,29394,49.0,(יפו ד' (גבעת התמרים,389591.6863,6631,49.0,2017-11-02 20:26:47,2017-11-02 04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136329,71-822-55,219682,שריון חינמי,7,40.33,"הרב יוסף צוברי 21, תל אביב יפו, ישראל","דרך ההגנה 124, תל אביב יפו, ישראל",2018-04-12 18:11:51,2018-04-12 19:01:09.560,10.414084,...,נוה אליעזר וכפר שלם מזרח,3.307963e+05,6693,68.0,"נוה ברבור, כפר שלם מערב",239869.3610,5066,68.0,2018-04-13 10:09:02,2018-04-13 02:00
136329,71-822-55,219682,שריון חינמי,7,40.33,"הרב יוסף צוברי 21, תל אביב יפו, ישראל","דרך ההגנה 124, תל אביב יפו, ישראל",2018-04-12 18:11:51,2018-04-12 19:01:09.560,10.414084,...,נוה אליעזר וכפר שלם מזרח,3.307963e+05,6693,68.0,"נוה ברבור, כפר שלם מערב",239869.3610,5066,68.0,2018-04-13 10:09:02,2018-04-12 23:00
136329,71-822-55,219682,שריון חינמי,7,40.33,"הרב יוסף צוברי 21, תל אביב יפו, ישראל","דרך ההגנה 124, תל אביב יפו, ישראל",2018-04-12 18:11:51,2018-04-12 19:01:09.560,10.414084,...,נוה אליעזר וכפר שלם מזרח,3.307963e+05,6693,68.0,"נוה ברבור, כפר שלם מערב",239869.3610,5066,68.0,2018-04-13 10:09:02,2018-04-13 07:00
136329,71-822-55,219682,שריון חינמי,7,40.33,"הרב יוסף צוברי 21, תל אביב יפו, ישראל","דרך ההגנה 124, תל אביב יפו, ישראל",2018-04-12 18:11:51,2018-04-12 19:01:09.560,10.414084,...,נוה אליעזר וכפר שלם מזרח,3.307963e+05,6693,68.0,"נוה ברבור, כפר שלם מערב",239869.3610,5066,68.0,2018-04-13 10:09:02,2018-04-12 22:00


In [501]:
r[r.hour_list.apply(lambda c: c == np.nan)]


,vehicle_id,order_id,shiryun_cat,avg_dist,billing_minutes,orig_addr,dest_addr,start_dt,end_dt,kmh,...,orig_nei_name,orig_nei_area,orig_nei_popu,dest_nei_id,dest_nei_name,dest_nei_area,dest_nei_popu,next_origin_nei_id,next_start_dt,hour_list


In [528]:
r = r.rename(columns={"hour_list": "stay_in_dest"})

In [531]:
r[["start_dt","origin_nei_id" ,"end_dt","dest_nei_id", "next_start_dt", "next_origin_nei_id","stay_in_dest","start_hour"]]

,start_dt,origin_nei_id,end_dt,dest_nei_id,next_start_dt,next_origin_nei_id,stay_in_dest,start_hour
0,2017-10-23 00:08:53,31.0,2017-10-23 00:31:40.370,49.0,2017-10-24 15:58:11,35.0,2017-10-23 00:31:40.370000,0
1,2017-10-24 04:01:12,31.0,2017-10-24 04:24:45.240,49.0,2017-10-28 20:58:25,46.0,2017-10-24 04:24:45.240000,4
2,2017-11-02 02:19:45,31.0,2017-11-02 02:41:30.513,49.0,2017-11-02 20:26:47,49.0,2017-11-02 05:00,2
2,2017-11-02 02:19:45,31.0,2017-11-02 02:41:30.513,49.0,2017-11-02 20:26:47,49.0,2017-11-02 20:00,2
2,2017-11-02 02:19:45,31.0,2017-11-02 02:41:30.513,49.0,2017-11-02 20:26:47,49.0,2017-11-02 04:00,2
...,...,...,...,...,...,...,...,...
136329,2018-04-12 18:11:51,69.0,2018-04-12 19:01:09.560,68.0,2018-04-13 10:09:02,68.0,2018-04-13 02:00,18
136329,2018-04-12 18:11:51,69.0,2018-04-12 19:01:09.560,68.0,2018-04-13 10:09:02,68.0,2018-04-12 23:00,18
136329,2018-04-12 18:11:51,69.0,2018-04-12 19:01:09.560,68.0,2018-04-13 10:09:02,68.0,2018-04-13 07:00,18
136329,2018-04-12 18:11:51,69.0,2018-04-12 19:01:09.560,68.0,2018-04-13 10:09:02,68.0,2018-04-12 22:00,18


In [532]:
a = 878695 

In [533]:
r.to_csv('data/df_stay_in_dest.csv', index=False,encoding='utf-8-sig' )